In [1]:
#ORPO Odds Ratio Preference Optimization
#aligning llm technique reduces the need of reference model
#DPO requires the need of reference model
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"



  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-gsui96m8/unsloth_d9ac9a04ee034b98bdc96d59183bbfb6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-gsui96m8/unsloth_d9ac9a04ee034b98bdc96d59183bbfb6
  Resolved https://github.com/unslothai/unsloth.git to commit 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:

!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-f0aqhnot
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-f0aqhnot
  Resolved https://github.com/huggingface/transformers.git to commit 6bd511a45a58eb02bd59cf447141a2af428747a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9131474 sha256=664a61e018855e7dc52a80fa9d50f91c9a67e3ba73c3af8227e446b012d111f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-bh67stoc/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.1
    Uninstalling transformers-4.41.1:
      Successfully uninstalled transformers-4.41.1


In [3]:

!pip install trl

In [5]:

from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:80: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [6]:

from datasets import load_dataset

In [7]:
max_seq_length = 2048

In [8]:
dataset = load_dataset("imdb", split="train")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit=True
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length
)


Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "unsloth-test",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.606300
2,2.343600
3,2.341700
4,2.476600
5,2.538100
6,2.685500
7,2.351200
8,2.369800
9,2.221000
10,2.537900


TrainOutput(global_step=60, training_loss=2.4146126667658487, metrics={'train_runtime': 707.0025, 'train_samples_per_second': 0.679, 'train_steps_per_second': 0.085, 'total_flos': 9642624560529408.0, 'train_loss': 2.4146126667658487, 'epoch': 0.0192})

In [13]:
inputs = tokenizer(
    [
      "I really liked the movie because it shows emotions and talks humanity."
    ],
    return_tensors="pt",
).to("cuda")

In [14]:

inputs

{'input_ids': tensor([[    1,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [15]:

outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:

outputs

tensor([[    1,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
           304,   315,  6557,   378,   298,  3376, 28723,   661,   349,   264,
          1215,  1179,  5994,   304,   315,  6557,   378,   298,  3376, 28723,
           661,   349,   264,  1215,  1179,  5994,   304,   315,  6557,   378,
           298,  3376, 28723,   661,   349,   264,  1215,  1179,  5994,   304,
           315,  6557,   378,   298,  3376, 28723,   661,   349,   264,  1215,
          1179,  5994,   304,   315,  6557,   378,   298,  3376, 28723,   661,
           349,   264,  1215,  1179,  5994,   304,   315,  6557,   378,   298,
          3376, 28723,   661,   349,   264,  1215,  1179,  5994,   304,   315,
          6557,   378,   298,  3376, 28723,   661,   349,   264,  1215,  1179,
          5994,   304,   315,  6557,   378,   298,  3376, 28723,   661,   349,
           264,  1215,  1179,  5994,   304,   315,  

In [17]:

tokenizer.batch_decode(outputs)

['<s> I really liked the movie because it shows emotions and talks humanity. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to']

In [18]:
model.save_pretrained("lora_model")

In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [20]:

!pip install huggingface_hub


In [24]:

from huggingface_hub import notebook_login

In [26]:
notebook_login()


In [27]:
model.push_to_hub("skuma307/unsloth_4bit_mistral_imdb_model")

HfHubHTTPError:  (Request ID: Root=1-6658e446-6e6b23355d17418900f0cadb;58bab1d0-c52d-4193-81b6-b42a2d6966a6)

403 Forbidden: You have read access but not the required permissions for this operation.
Cannot access content at: https://huggingface.co/skuma307/unsloth_4bit_mistral_imdb_model.git/info/lfs/objects/batch.
If you are trying to create or update content,make sure you have a token with the `write` role.